<h2>Trabalho de Análise de Sentimentos   </h2>
<br>
<b>Equipe: Igor, Jane, Jerusa e Leila   <br>
  O artigo completo sobre Repeated k-Fold Cross-Validation pode ser encontrado neste site 
<a href='https://machinelearningmastery.com/repeated-k-fold-cross-validation-with-python/'> neste dataset </a><br>
    

<h5>Versão 12/09/2020 - Criada a função Trata_Emojis() que traduz o sentimento da imagem.<br>
    Versão 27/08/2020 - Corrigido arquivos com neutras e sem neutras.<br>
    Versão 10/08/2020 - Organizado em funções e com o repeat k-fold. </h5>

In [1]:
# Informando o número desejado de repetições para K-fold
nrMaxTestes = 5

In [2]:
import pandas as pd
import chardet
import matplotlib.pyplot as plt
import emoji
import regex
import nltk
nltk.download('stopwords')
nltk.download('rslp')
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble     import RandomForestClassifier
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm          import LinearSVC
from sklearn.cluster      import KMeans
from sklearn.multiclass   import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.datasets        import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import LinearSVC
from matplotlib              import pyplot
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from sklearn import utils
from sklearn.preprocessing import MinMaxScaler

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\particular\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\particular\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [3]:
%matplotlib inline   
def Carrega_Arquivo(plota_grafico):
    #arquivo_f = 'Suicidas_NaoSuicidas_Testar-20200521.csv'
    arquivo_f = 'Suicidas_NaoSuicidas_Testar-20200718.csv'
    cabecalho_f = ['Polaridade', 'Frase']
    with open(arquivo_f, 'rb') as f:
        result_f = chardet.detect(f.read()) 
    data = pd.read_csv(arquivo_f, encoding=result_f['encoding'], sep=';', header=None, names=cabecalho_f)
    
    # Excluindo os registros com frase nulas
    data = data[pd.notnull(data['Frase'])]

    # criou a coluna Polaridade_id no dataframe df, que contém o nome polaridade representado por número
    data['Polaridade_id'] = data['Polaridade'].factorize()[0]

    if plota_grafico == 'SIM':
        data['Polaridade'].value_counts().plot(kind='pie');
        plt.title('ESSA É A DISTRIBUIÇÃO DOS DADOS DE ENTRADA')
        plt.savefig("distribuicao_dados.png")

    return(data)

In [4]:
def Carrega_Parametros():
    efile = open("parametros.txt", "r")
    parametros = []
    for nr, linha in enumerate(efile):
        conteudo = linha.split(";") 
        parametros.append(conteudo) 
    efile.close()
    return(parametros)

In [5]:
def Limpa_emojis(linha):
    
    data = regex.findall(r'\X', linha)
    for word in data:        
        if any(char in emoji.UNICODE_EMOJI for char in word):
            # Remove emoji do texto
            
        
    return (linha)

In [19]:
# https://emojipedia.org/people/
# https://www.emoticonsignificado.com.br/lista-emojis-pessoas-whatsapp
def Trata_Emojis(linha):
    # Dicionário de emojis e emoções
    emojis_lista = [('😊', ' feliz '), 
                    ('🤣', ' super feliz '), 
                    ('😂', ' achando engraçado demais ')
                    ('😢', ' triste ')
                    ('😭', ' muito triste ')
                    ('😡', ' furioso ')
                    ('😱', ' assustado ')
                    ('😩', ' chateado ')
                    ('😰', ' ansioso ')
                    ('🤩', ' encantado')]
    emojis_dict = dict(emojis_lista)
    
    # quebra a linha em caracteres 
    linha_quebrada = regex.findall(r'\X', linha)
     
    for char in linha_quebrada:
        if char in emoji.UNICODE_EMOJI:
            char_demojize = emoji.demojize(char)
            y = emoji.emojize(char_demojize)
            emoji_sentimento =  emojis_dict.get(y, ' ')
            linha = linha.replace(char, emoji_sentimento)  
         
    return(linha)
    


    Of course, (too many) emoji [a lot] characters      feliz  like  , #@^!*&#@^#   helps   people read  achando engraçado demais aa achando engraçado demais aaa achando engraçado demais a #douchebag
    


In [7]:
def Carrega_Stopwords():
    stopwords_nltk = set(nltk.corpus.stopwords.words('portuguese'))
    f = open('stop_palavras.txt', 'r', encoding = 'utf-8')
    tudo =  f.read()
    palavras =  set(tudo.split())
    nltk_mais_palavras = stopwords_nltk.union(palavras)
    return(nltk_mais_palavras)

In [12]:
def Limpa_Texto(instancia):
    instancia = instancia.lower()  
    acentos = str.maketrans("áéíóúâêôãõ", "aeiouaeoao") 
    instancia = instancia.translate(acentos)
    
    instancia = Trata_Emojis(instancia)
    instancia = re.sub(r'\|\|\|', r'', instancia) 
    instancia = re.sub(r'http\S+', r'', instancia)
    instancia = instancia.replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','')    
    instancia = instancia.replace('[','').replace(']','').replace('@','').replace('(','').replace('/','')   
    instancia = instancia.replace('#','').replace('RT @','').replace('rt cvvoficial','')
    instancia = instancia.replace('!','').replace('?','')
   
    stopwords = Carrega_Stopwords()
    palavras = []
    for i in instancia.split():
        if not i.isnumeric(): 
            if not i in stopwords:
                palavras.append(i)
        
               
    return (" ".join(palavras))

In [ ]:
def Aplica_Stemizacao(instancia):
    palavras = [stemmer.stem(i)  for i in instancia.split()]
    return (" ".join(palavras))

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [ ]:
#!pip install nltk

In [ ]:
def Transforma_Doc2vec(t_frases, t_polaridades):
    
    df_teste = pd.DataFrame({'Frase': t_frases, 'Polaridade_id':t_polaridades})
    df_teste['tokenized'] = df_teste.apply(lambda row: nltk.word_tokenize(row['Frase']), axis=1)

    train_tagged = df_teste.apply(
            lambda r: TaggedDocument(r.tokenized, tags=[r.Polaridade_id]), axis=1)
    
    model_dmm = Doc2Vec(dm=1, vector_size=300, window=3, min_count=2, workers=cores)
    model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

    for epoch in range(10):
        model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
        model_dmm.alpha -= 0.002
        model_dmm.min_alpha = model_dmm.alpha

    y_train, X_train = vec_for_learning(model_dmm, train_tagged)

    # adapta as features a uma escala de [0 até 1]
    scaler = MinMaxScaler()
    scaled_X_train = scaler.fit_transform(list(X_train))

    return(scaled_X_train, y_train)
    

In [ ]:
def Carrega_features_labels(tipo_NLP, q_frases, q_polaridades ):
    
    rotulos = q_polaridades
      
    if tipo_NLP == 'CountVectorizer':
        v = CountVectorizer()
        vetores = v.fit_transform(q_frases) 
        
    if tipo_NLP == 'TF_IDF':
        tfidf = TfidfVectorizer(use_idf=True, max_features=300, max_df=5, min_df=1, norm='l2', encoding='utf-8', ngram_range=(1,2))
        vetores = tfidf.fit_transform(q_frases) 
        
    if tipo_NLP == 'DOC2VEC':
        vetores, rotulos  = Transforma_Doc2vec(q_frases, q_polaridades)
    
 
    return(vetores, rotulos)
 

In [ ]:
def Calcula_Acuracia_Modelos(features, labels, MaxTeste, tipo_NLP, resultados, com_stemizacao, com_neutras):
    
    
    models = [
        RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
        LinearSVC(),
        #OneVsRestClassifier(LinearSVC(), n_jobs=1),
        MultinomialNB(),
        LogisticRegression(random_state=0) ]
    
    
    
    print('--------------------------------------------------------------------------------------')
    print('Tipo Conversao NLP=> %s - stemizacao= %s e neutras=%s' %  (tipo_NLP, com_stemizacao, com_neutras))
    print('--------------------------------------------------------------------------------------')
         
    
    for model in models:
        model_name = model.__class__.__name__
        
        
        results = []
        numero = 0 
        for repeats in range(1,MaxTeste):
            CV = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
            # n_jobs = -1 significa usar todos os processadores
            scores = cross_val_score(model, features, labels, scoring='accuracy', cv=CV, n_jobs=-1)
            print('Modelo %s > Repetições=%d média=%.4f desvio padrão=%.3f' % (model_name, repeats, mean(scores), sem(scores)))
           
            # guarda o resultado
            results.append(scores)
            numero = repeats * 10 
            #numero = int((numero + 1) * 10 )
            resultados.append((tipo_NLP, com_stemizacao, testa_com_Neutra, numero, model_name, mean(scores)))
            
        # cria gráfico dos resultados
        #pyplot.boxplot(results, labels=[str(r) for r in range(1,MaxTeste)], showmeans=True)
        #pyplot.title(model_name)
        #pyplot.show()
        
    return()
       

In [ ]:
def Mostrar_max_por_modelo(modelo, data_teste):
 
    eh_modelo =  data_teste['tipo_NLP']==modelo
    data_filtrada = data_teste[eh_modelo]
    
    max_value = data_filtrada['media_acuracia'].max()
    eh_maior  = data_filtrada['media_acuracia']==max_value
    data_filtrada = data_filtrada[eh_maior]
    return(data_filtrada) 


In [ ]:
def Mostra_Resultados(Nome_Arquivo):

    data_mostra = pd.read_csv(Nome_Arquivo, sep=",")

    # mostra execução que apresentou a maior acurácia
    maximo = data_mostra['media_acuracia'].idxmax()
    rowData = data_mostra.loc[maximo , : ]
    linha_com_maximo = rowData.values.tolist()
    df_teste = pd.DataFrame([linha_com_maximo])
    df_teste.columns = ['nr','tipo_NLP', 'com_stemizacao', 'testa_com_Neutra', 'repeticoes', 'model_name', 'media_acuracia']
   
    # mostra Máxima Acurácia por Modelo = 
    teste1 = Mostrar_max_por_modelo('CountVectorizer', data_mostra)
    teste2 = Mostrar_max_por_modelo('TF_IDF', data_mostra)
    teste3 = Mostrar_max_por_modelo('DOC2VEC', data_mostra)
    teste_r = pd.concat([teste1, teste2, teste3])
    teste_r.columns = ['nr','tipo_NLP', 'com_stemizacao', 'testa_com_Neutra', 'repeticoes', 'model_name', 'media_acuracia']
    
    # mostra Valor Medio de Acuracia nas Iterações
    Valor_Medio_Acuracia = data_mostra.groupby(['tipo_NLP', 'model_name']).agg({'media_acuracia': ['mean']})
    Valor_Medio_Acuracia.columns = [ 'Acurácia Média']
     
    return(df_teste, teste_r, Valor_Medio_Acuracia)

In [ ]:
def Mostra_Grafico(Nome_Arquivo):
    # li dados
    data_grafico = pd.read_csv(Nome_Arquivo, sep=",")
    data_grafico.columns = ['nr','tipo_NLP', 'com_stemizacao', 'testa_com_Neutra', 'repeticoes', 'model_name', 'media_acuracia']
    
    # cria um DataFrame Pandas com resultado do filtro
    result = data_grafico.groupby(['tipo_NLP','model_name'])[['media_acuracia']].mean()

    # gera uma lista com os valores de acurácia 
    valores = list(result.media_acuracia) 

    labels = list(np.unique(data_grafico.model_name))

    # os 4 primeiros na lista de valores são do countvectorizer
    medias_countvectorizer = valores[0:4]
    medias_tfidf = valores[4:8]
    medias_doc2vec = valores[8:12]

    # Definindo a largura das barras
    barWidth = 0.25

    # Aumentando o gráfico
    plt.figure(figsize=(10,5))

    # Definindo a posição das barras
    r1 = np.arange(len(medias_countvectorizer))
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]

    # Criando as barras
    plt.bar(r1, medias_countvectorizer, color='#6A5ACD', width=barWidth, label='CountVectorizer')
    plt.bar(r2, medias_tfidf, color='#00BFFF', width=barWidth, label='TF-IDF')
    plt.bar(r3, medias_doc2vec, color='#48F317', width=barWidth, label='DOC2VEC')

    # Adiciando legendas as barras
    plt.xlabel('Modelos')
    plt.xticks([r + barWidth for r in range(len(medias_countvectorizer))], labels)
    plt.ylabel('Acurácia Média')
    plt.title('Representação Acurácia Média por Modelo')
        

    # Criando a legenda e exibindo o gráfico
    plt.legend()
    plt.savefig("acuracia_por_modelo.png")
    plt.show()
    
    return()

In [ ]:
def Mostra_Grafico_por_Opcao(Nome_Arquivo, opcao):
    
    arquivo_res = pd.read_csv(Nome_Arquivo, sep=",")
    arquivo_res.columns = ['nr','tipo_NLP', 'com_stemizacao', 'testa_com_Neutra', 'repeticoes', 'model_name', 'media_acuracia']
    
    if opcao == 'SEM_NEUTRAS':
        data_grafico = arquivo_res[arquivo_res['testa_com_Neutra'] == 'NAO']
        titulo = 'Sem Incluir Frases Neutras'
        nome_grafico = 'sem_neutras.png'
    elif opcao == "COM_STEMMIZACAO":
        data_grafico = arquivo_res[arquivo_res['com_stemizacao'] == 'SIM']
        titulo = 'Utilizando a Stemmização'
        nome_grafico = 'com_stemizacao.png'
        
    # cria um DataFrame Pandas com resultado do filtro
    result = data_grafico.groupby(['tipo_NLP','model_name'])[['media_acuracia']].mean()

    # gera uma lista com os valores de acurácia 
    valores = list(result.media_acuracia) 

    labels = list(np.unique(data_grafico.model_name))
   
    # os 4 primeiros na lista de valores são do countvectorizer
    medias_countvectorizer = valores[0:4]
    medias_tfidf = valores[4:8]
    medias_doc2vec = valores[8:12]

    # Definindo a largura das barras
    barWidth = 0.25

    # Aumentando o gráfico
    plt.figure(figsize=(10,5))

    # Definindo a posição das barras
    r1 = np.arange(len(medias_countvectorizer))
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]

    # Criando as barras
    plt.bar(r1, medias_countvectorizer, color='#6A5ACD', width=barWidth, label='CountVectorizer')
    plt.bar(r2, medias_tfidf, color='#00BFFF', width=barWidth, label='TF-IDF')
    plt.bar(r3, medias_doc2vec, color='#48F317', width=barWidth, label='DOC2VEC')

    # Adiciando legendas as barras
    plt.xlabel('Modelos')
    plt.xticks([r + barWidth for r in range(len(medias_countvectorizer))], labels)
    plt.ylabel('Acurácia Média')
    plt.title(titulo)

    # Criando a legenda e exibindo o gráfico
    plt.legend()
    plt.savefig(nome_grafico)
    plt.show()

    return()

In [ ]:
def Grava_Resultados(resultados):
    NomArq = 'resultados_'  + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.csv'
    resultados_df = pd.DataFrame(resultados, columns=['tipo_NLP', 'com_stemizacao', 'testa_com_Neutra','repeticoes', 'model_name', 'media_acuracia'])
    resultados_df.to_csv(NomArq)
    return(NomArq)

# INICIO DO PROCESSAMENTO 

In [ ]:
data = Carrega_Arquivo('SIM')

In [ ]:
# Aplica a função de limpeza de caracteres em todos os dados:
data.Frase = [Limpa_Texto(i) for i in data.Frase]

# cria coluna com frases stemizadas 
stemmer = nltk.stem.RSLPStemmer()
data['Frase_Stemm'] = [Aplica_Stemizacao(j) for j in data.Frase]
#data.head()

In [ ]:
filtro  = data['Polaridade'] != 'Neutra'
data_sem_neutras = data[filtro]
#data_sem_neutras.head()

In [ ]:
# frases limpas são salvas para uso em outros programas.
import pickle
import os 

# Salvando dataframe para usar em Pipe_Doc2Vec
pickle.dump(data, open("dados_treino.pickle", 'wb'))

# Salvando frases para usar no K-Means
frases_file = 'Frases_Limpas.pickle'
pickle.dump(data.Frase, open(frases_file, 'wb'))


In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

## explicação sobre K-repeated e K-fold está na última célula

In [ ]:
# Busca definições para parâmetros
parametros = Carrega_Parametros()

# inicializa lista de resultados
resultados = []

for lin in range(1,len(parametros)):

    tipo_NLP         = parametros[lin][0]
    com_stemizacao   = parametros[lin][1]
    testa_com_Neutra = parametros[lin][2]
    com_stemizacao   = com_stemizacao.strip()
    testa_com_Neutra = testa_com_Neutra.strip()

    x_frases = []
    if testa_com_Neutra == 'SIM':
        x_polaridades = data.Polaridade_id 
        
        if com_stemizacao == 'SIM':
            x_frases = data.Frase_Stemm
            
        if com_stemizacao == 'NAO':
            x_frases = data.Frase
            

    if testa_com_Neutra == 'NAO':
        x_polaridades = data_sem_neutras.Polaridade_id 
        
        if com_stemizacao == 'SIM':
            x_frases = data_sem_neutras.Frase_Stemm
           
        if com_stemizacao == 'NAO':
            x_frases = data_sem_neutras.Frase 
    

    
    features, labels = Carrega_features_labels(tipo_NLP, x_frases, x_polaridades)
      
    Calcula_Acuracia_Modelos(features, labels, nrMaxTestes, tipo_NLP, resultados, com_stemizacao, testa_com_Neutra)

# salva os resultados
print('*** Salvando Resultados ***')
Nome_Arquivo = Grava_Resultados(resultados)

In [ ]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [ ]:
# mostra os resultados
df1, df2, df3 = Mostra_Resultados(Nome_Arquivo)
printmd('**Maior Acurácia nos Testes**')
df1

In [ ]:
printmd('**Máxima Acurácia por Modelo**')
df2

In [ ]:
printmd('**Valor Medio de Acuracia nas Iterações**')
df3

In [ ]:
# apresenta gráfico contendo as médias de acurácia encontradas
Mostra_Grafico(Nome_Arquivo)
Mostra_Grafico_por_Opcao(Nome_Arquivo, "SEM_NEUTRAS")
Mostra_Grafico_por_Opcao(Nome_Arquivo, "COM_STEMMIZACAO")

<h2> ==> Escolhemos manualmente o modelo que apresentou melhor performance <br>
    <br>

 
 ## Observando resultados acima vimos que:
 <h3><i> Os resultados mostram que o LinearSVC apresentou melhor performance,tanto com TF-IDF quanto DOC2VEC:<br>
    -     LinearSVC                     0.780115  (TFIDF)<br>   
    -     LinearSVC                     0.769629 (DOC2VEC) <br>
 o Multinomial foi o melhor para CountVectorizer. <br>
 Além disso, observamos que a melhor acurácia aconteceu quando  NAO usamos Neutras e NAO usamos stemização.
 

- A maior acurácia foi igual a 0.8498508098891732, utilizando o modelo Multinomial, usando 'CountVectorizer'.<br>

- Embora o CountVectorizer tenha apresentado melhor acurácia, o método é considerado menos <br>
  eficiente que o TF-IDF, por essa razão estamos elegendo o que apresentou melhor desempenho no TF-IDF.
  
- A seguir treinaremos o LinearSVC, com TF-IDF, sem stemização e sem frases Neutras. </i></h3>

<h2><i><b>Validação cruzada do k-Fold</b>

In [ ]:
# pegando o arquivo sem usar frases neutras e a coluna 'Frase'  que não tem stemmização
# não incluir neutras => usar  data_sem_neutras
# incluir stemmização => usar data.Frase_Stemm
data.head()
data.shape

In [ ]:
# evaluate a linearSVC model using k-fold cross-validation
# https://machinelearningmastery.com/repeated-k-fold-cross-validation-with-python/
from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.datasets        import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import LinearSVC
from matplotlib              import pyplot

#tfidf = TfidfVectorizer(use_idf=True, max_features=300, max_df=5, min_df=1, norm='l2', encoding='utf-8', ngram_range=(1,2))
tfidf = TfidfVectorizer(use_idf=True, max_df=5, min_df=1, norm='l2', encoding='utf-8', ngram_range=(1,2))

# prepara labels e features
#features = tfidf.fit_transform(data_sem_neutras.Frase)
#features = tfidf.fit_transform(data_sem_neutras.Frase_Stemm)
#labels = data_sem_neutras.Polaridade_id
features = tfidf.fit_transform(data.Frase)
#features = tfidf.fit_transform(data.Frase_Stemm)
labels = data.Polaridade_id

In [ ]:
#Cria matriz para mostrar palavras utilizadas 
matriz_tfidf = pd.DataFrame(features.toarray(), columns=tfidf.get_feature_names())
matriz_tfidf.head()

In [ ]:
# avalia o modelo com um dado nr de repetições
def evaluate_model(X, y, repeats):
    # prepara a cross-validation  
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # cria o modelo
    model = LinearSVC()
    # avalia o modelo
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return(scores)  

# configurações de teste
results = list()

for r in range(1,16):
    scores = evaluate_model(features, labels, r)
    print('LinearSVC > Repetições=%d média=%.4f desvio padrão=%.3f ' % (r, mean(scores), sem(scores)))
    
    # guarda o resultado
    results.append(scores)
    

In [ ]:
# cria gráfico dos resultados
pyplot.boxplot(results, labels=[str(r) for r in range(1,16)], showmeans=True)
pyplot.show()

 ## Analisando resultados:
<i>
- A linha laranja indica a mediana da distribuição e o triângulo verde representa a média aritmética. Na sexta repetição vemos que  os valores coincidem, e isso sugere uma distribuição simétrica bem razoável, e que a média conseguiu capturar bem a tendência central. Isso nos ajuda a escolher o número apropriado de repetições para o processo de teste. <br>
Usaremos 6 repetições.  


In [ ]:
features.shape

In [ ]:
# Treinando o LinearSVC utilizando TF-IDF
from sklearn.model_selection import train_test_split
from sklearn.svm             import LinearSVC 

model = LinearSVC()

# Separando dados de treino e teste 
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.10, random_state=0)


# Treina o modelo
model.fit(X_train, y_train)
 

Score = model.score(X_test,y_test) 
print('Score=', Score)


### Salvando o Modelo Vencedor

In [ ]:
# O Modelo é salvo para classificar twitters no programa Analise_Twitters_CVV.ipynb.
import pickle
import os 

#os.chdir('C:\\Users\\particular\\SERIQUE_TCC\\FINAL DA POS\\')

# Salvando modelo
model_file = 'model_LinearSVC.pickle'
pickle.dump(model, open(model_file, 'wb'))

# Salvando tfidf
tfidf_file = 'tfidf_file.pickle'
pickle.dump(tfidf, open(tfidf_file, 'wb'))

print("Dados salvos!!!")

In [ ]:
# Dicionário para polaridade  
id_to_Polaridade = {0: 'Negativa', 1: 'Positiva', 2: 'Neutra'}

texts = ["Estou em beco sem saida, o unico jeito é tomar todos comprimidos e dormir para sempre.",
         "Estranho que não sinto forças para levantar da cama. Preciso de ajuda.",
         "Adoro viver, cantar e sorrir todo dia.", 
          "Minha vida é um pesadelo imenso, estou muito triste.", 
          "O jogo foi adiado para amanhã."]
corretas = [1, 1, 0, 1, 2]

text_features = tfidf.transform(texts)

nr = 0 
acertou = 0 
predictions = model.predict(text_features)
for text, predicted in zip(texts, predictions):
    print('"{}"'.format(text))
    print("  - Classificada como: '{}'".format(id_to_Polaridade[predicted]))
    print("  - O correto é      : '{}'".format(id_to_Polaridade[corretas[nr]]))
    print("")
    if predicted == corretas[nr]:
        acertou += 1
    nr += 1
print('-' * 50)
print('acertou = ', acertou,  '/', len(texts)) 
print('-' * 50)

## CONCLUSÃO: A assertividade em frases que não estavam no modelo foi péssima, apesar do modelo ter sido considerado o melhor nos testes.

In [ ]:
print('*** A C A B E I ***')

<i><b> Descobrindo o melhor k-fold.</b>
O procedimento de validação cruzada com dobra k divide um conjunto de dados limitado em k dobras sem sobreposição. Cada uma das k dobras tem a oportunidade de ser usada como um conjunto de testes retidos, enquanto todas as outras dobras coletivamente são usadas como um conjunto de dados de treinamento. Um total de k modelos são ajustados e avaliados nos conjuntos de testes de espera k e o desempenho médio é relatado.
   
¨ A validação cruzada repetida em dobra k replica o procedimento […] várias vezes. Por exemplo, se a validação cruzada 10 vezes foi repetida cinco vezes, 50 conjuntos diferentes retidos seriam usados para estimar a eficácia do modelo.¨

- Página 70, Modelagem Preditiva Aplicada , 2013.
Applied Predictive Modeling 1st ed. 2013, Corr. 2nd printing 2018 Edition
by Max Kuhn  (Author), Kjell Johnson  (Author)
    
  
O desempenho médio relatado de uma única execução de validação cruzada k-fold pode ser barulhento.
A validação cruzada repetida de dobras k fornece uma maneira de reduzir o erro na estimativa do desempenho médio do modelo.
Como avaliar modelos de aprendizado de máquina usando a validação cruzada k-fold repetida no Python.

Nesse caso, podemos ver que o padrão de uma repetição parece otimista em comparação com os outros resultados, com uma precisão de cerca de 86,80 por cento em comparação com 86,73 por cento e menor com números diferentes de repetições.

Podemos ver que a média parece coalescer em torno de um valor de cerca de 86,5%. Podemos considerar isso como uma estimativa estável do desempenho do modelo e, por sua vez, escolher 5 ou 6 repetições que parecem aproximar esse valor primeiro.

Observando o erro padrão, podemos ver que ele diminui com um aumento no número de repetições e se estabiliza com um valor em torno de 0,003, em torno de 9 ou 10 repetições, embora 5 repetições atinjam um erro padrão de 0,005, metade da alcançada com um repetição única.

Iremos variar os seguintes parâmetros:<br>
Se dm = 0, é usado opacote distribuído de palavras (PV-DBOW) ; <br>
se dm = 1, 'memória distribuída' (PV-DM) é usada. <br>
Vetores de recursos de 100 dimensões.<br>
min_count = 2, ignora todas as palavras com frequência total inferior a esta. <br>
negativo = 5, especifica quantas “palavras de ruído” devem ser desenhadas. <br>
hs = 0, e negativo é diferente de zero, amostragem negativa será usada. <br>
amostra = 0, o limite para configurar quais palavras de alta frequência são amostradas aleatoriamente. <br>
trabalhadores = núcleos, use esses muitos threads de trabalho para treinar o modelo